<a href="https://colab.research.google.com/github/ArtyomGrachev/defeater/blob/fp-pointnet-generator/Frustrum_FP_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade pip
!apt-get install tree
!pip install importlib

In [0]:
# Import first part of train dataset from google cloud

from google.colab import auth
auth.authenticate_user()

In [0]:
#!gsutil cp 	gs://waymo_open_dataset_v_1_0_0/training/training_0001.tar /content/training_data_0001.tar

In [0]:
# copy train 0001 tar into google drive

#!gsutil cp gs://waymo_open_dataset_v_1_0_0/training/training_0001.tar /content/drive/'My Drive'/Waymo_OD/

In [0]:
# Clone github repo with util functions

!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od

Cloning into 'waymo-od'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 716 (delta 38), reused 44 (delta 15), pack-reused 618
Receiving objects: 100% (716/716), 14.19 MiB | 36.88 MiB/s, done.
Resolving deltas: 100% (421/421), done.


In [0]:
#!pip install -U tensorflow_estimator

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
# import python and waymo dataset libraries 

import sys
import tensorflow as tf

# fixes import error for python 2.7
sys.path.insert(1, r"/usr/local/lib/python3.6/dist-packages")

!pip install waymo-open-dataset

import os
import shutil
import inspect
import tqdm
import numpy as np

import itertools
import imghdr
import functools
import math
import tarfile
import pandas as pd
import collections
from pprint import pprint
import matplotlib.pyplot as plt

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset.utils import  box_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

# This cell requires to restart colab run-time after exec

### Waymo-OD to Kitti adapter

In [0]:
# Transform Waymo dataset style to KITTI dataset style
# slightly modified version of https://github.com/Yao-Shao/Waymo_Kitti_Adapter conversion program 

In [0]:
!python ./drive/'My Drive'/aid-submission/adapter_file_v3.py --source_dir ./drive/'My Drive'/Waymo_OD --target_dir ./drive/'My Drive'/Waymo_to_Kitti_adapter_train_1

### Setup frustum Point-net (YOLOv3 backbone) model for Waymo-OD

##### Setup YOLOv3

In [0]:
!cat ./drive/'My Drive'/aid-submission/setup.sh

#!/bin/bash

git clone https://github.com/wizyoung/YOLOv3_TensorFlow.git ./drive/'My Drive'/YOLOv3_TensorFlow/
wget https://pjreddie.com/media/files/yolov3.weights -P ./drive/'My Drive'/YOLOv3_TensorFlow/data/darknet_weights/
echo "" > ./drive/'My Drive'/YOLOv3_TensorFlow/__init__.py
cd ./drive/'My Drive'/YOLOv3_TensorFlow/
python convert_weight.py


In [0]:
# Download pretrained on ImageNet YOLOv3

!bash ./drive/'My Drive'/aid-submission/setup.sh

##### Setup frustum pointnet 

In [0]:
# Modules path managing

!pip install opencv-python
!cp -r ./drive/'My Drive'/YOLOv3_TensorFlow ./

sys.path.insert(1, "/usr/local/lib/python2.7/dist-packages/cv2")
sys.path.insert(1, "/content/drive/My Drive/")
sys.path.insert(1, r"./drive/My Drive/aid-submission")

In [0]:
# delete cached .pyc files for editing purposes
# !find ./drive/'My Drive'/aid-submission/ -name '*.pyc' -delete 

from collections import defaultdict
from matplotlib import cm
import cv2
from google.colab.patches import cv2_imshow

from data_loader import Parser
from detection2d import Detector2D
from detection3d import Detector3D

import messages
from messages import Detection2d, Detection3d
from transformer import Transformer
from frustums import single_frustum_iterator
from training.pointnet import find_person_position

W0330 21:45:40.153280 140426287335296 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
class frustum_FP_sampler(object):
  """Samples FP pedestrian detections via frustum pointnet"""

  def __init__(self, model_path, dataset_path, labled_mode_flag):
    """Initialize folder path and setup input parser."""
    self.dataset_path = dataset_path
    self.labled_mode_flag = labled_mode_flag
    self.parser = Parser(dataset_path, labled_mode_flag)
    self.fp_pedestrian = []

    self.GT_count = 0
    
    self.detector_2d = Detector2D()
    self.detector_3d = Detector3D(model_path)

    self.cmap = plt.get_cmap('jet')
    

  def run_detector(self, num_frames, camera_id=2, plot_det=False, fp_min_pts_num_threshold=75, fp_intersection_ratio=1./2., dest_path=None):
    """
    Performs FP detections on data from parser.
    Arguments:
      num_frames (int): set equals to -1 for full dataset parsing

      camera_id (int from 0 to 4): what camera should be used for 2D YOLO detection

      plot_det (bool): visualize detection (image + 2D-bbox + lidar points segmented as pedestrians)

      fp_min_pts_num_threshold (int): minimal number of points in frustum (that are classified as pedestrians) to consider it a valid classification
      (e.g. if there are only 10 points segmented as pedestrian its not a valid detection)

      fp_intersection_ratio (float from 0 to 1): minimal fraction of correctly segmented points in frustum to consider it a TP detection

      dest_path (NoneType or string): Where false positive detection must be saved as .npy binary files
    """

    assert type(num_frames) == int
    assert camera_id in {0,1,2,3,4}
    assert type(plot_det) == bool
    assert type(fp_min_pts_num_threshold) == int
    assert type(fp_intersection_ratio) == float and (0 < fp_intersection_ratio < 1)
    assert dest_path is None or type(dest_path) == str

    max_frame_num = float('inf') if num_frames == -1 else num_frames
    
    if dest_path is not None and not os.path.exists(dest_path):
      os.mkdir(dest_path)

    self.current_iter_id = 0 
    self.frame_number = 0
    
    # DEBUG:
    # self.test_frustum_points = []

    with self.detector_2d:
      for image, velo, calib, labels_ in self._labled_parser(self.parser):
        print("Frame number: {0}".format(self.frame_number))
      
        GT_pedestrians_box_3D, GT_pedestrians_box_2D = self._parse_labels_to_boxes(labels_)

        self.GT_count += len(GT_pedestrians_box_3D)

        boxes, scores, labels = self.detector_2d(image[:, :, :3])

        detections_2d = self._convert_yolo_2_detections(boxes, scores, labels)

        # REMIDER: update velo (it will have NLZ tags as 4-th column in it) 
        detections_3d = self.detector_3d(image, velo, calib, detections_2d)
        
        transformer = Transformer(calib)

        if plot_det: # render GT bboxes and YOLO bboxes
          projected_points, mask = transformer.project_points(velo[:, :3], image.shape,
                                                              return_only_valid=False)
          for gt in GT_pedestrians_box_2D:
            cv2.rectangle(image, (int(gt[0,0]), int(gt[1,1])), (int(gt[1,0]), int(gt[0,1])), (0, 0, 255, 255), 3)

          for d in detections_2d:
            cv2.rectangle(image, (int(d.x_min), int(d.y_min)), (int(d.x_max), int(d.y_max)), (0, 255, 0, 255), 3)
        

        vtc_tr_matrix = calib["Tr_velo_to_cam_{0}".format(camera_id)].reshape(4,4) # Get calibration matrix for required camera 
        inv_vtc_transform_matrix = np.linalg.inv(vtc_tr_matrix)[:3, :3].T          # Inverse matrix (project from sensor frame to vehicle frame) 


        for frustum_points, frustum_mask in single_frustum_iterator(image, velo, transformer, detections_2d):

            point_labels_mask = (self.detector_3d.pedestrian_model.predict([frustum_points, 
                                                                        np.zeros(frustum_points.shape[0], dtype=np.int32)])[0] > 0)

            masked_points = frustum_points[point_labels_mask]
            # Transform lidar points to vehicle frame
            frustum_points_vc = np.dot(masked_points - vtc_tr_matrix[:3, 3], inv_vtc_transform_matrix)
            
            
            if len(masked_points) < fp_min_pts_num_threshold: # not enough segmented points in frustum
              print("Not enough points in frustum. Skipped.")
              continue
            
            print(self.current_iter_id, len(GT_pedestrians_box_3D))
            if len(GT_pedestrians_box_3D) > 0:
              max_intersection_pts = np.max(
                                            box_utils.compute_num_points_in_box_3d(
                                                          tf.constant(frustum_points_vc), 
                                                          tf.constant(GT_pedestrians_box_3D)).eval(session=self.detector_2d.session)
                                            )
              # True positive detection (=> skip to next detection)
              if max_intersection_pts > fp_intersection_ratio * len(masked_points):
                print("True positive detection.")
                continue              

            #self.test_frustum_points.append(frustum_points_vc)

            print("FP detections of size {0} created: {1}/{2}".format(len(frustum_points_vc), self.current_iter_id + 1, max_frame_num))

            self.current_iter_id += 1
            if dest_path:
              np.save(os.path.join(dest_path, "{0}.npy".format(self.current_iter_id).zfill(12)), frustum_points_vc)


            # segmented points variation
            if plot_det:             
              for p in projected_points[frustum_mask][point_labels_mask]:
                color = tuple(int(255 * x) for x in self.cmap(p[2] / 40.0)[:3])
                cv2.circle(image, (int(p[0]), int(p[1])), 4, color, -1)

        if plot_det:
          pprint(labels_)
          cv2_imshow(cv2.resize(image, (820, 820)))
        
        self.frame_number += 1

        if max_frame_num <= self.current_iter_id:
            break


  def get_fp_point_cloud_size(self):
    return list(map(len,  self.fp_pedestrian))


  def _parse_labels_to_boxes(self, labels):
    boxes_2D = []
    boxes_3D = []

    for gt in labels:
      if gt['type'] == "PEDESTRIAN":
        boxes_2D.append(gt['bbox'].reshape(2,2))
        boxes_3D.append(np.hstack([gt['location'], np.flip(gt['dimensions']), gt['rotation_y']]))

    return np.array(boxes_3D), np.array(boxes_2D)


  def _labled_parser(self, parser):
    if self.labled_mode_flag:
        for image, velo, calib, labels in parser.all:
            yield image, velo, calib, labels
    else:
        for image, velo, calib in parser.all:
            yield image, velo, calib, []


  def _convert_yolo_2_detections(self, boxes, scores, labels):
    detections_2d = []
    for box, score, label in zip(boxes, scores, labels):
        if label != 0:  # person
            continue
        d = Detection2d()
        d.x_min = box[0]
        d.y_min = box[1]
        d.x_max = box[2]
        d.y_max = box[3]
        d.label = messages.pedestrian
        d.score = score
        detections_2d.append(d)
    return detections_2d

In [0]:
MODEL_PATH = "./drive/My Drive/aid-submission/models/pedestrian"
WAYMO_PATH = "./drive/My Drive/Waymo_to_Kitti_adapter_train_1/"
LABLED_MODE_FLAG = True

FP_sampler = frustum_FP_sampler(MODEL_PATH, WAYMO_PATH, LABLED_MODE_FLAG)

In [0]:
def fill_with_empty_lables(waymo_path, label_folder, image_folder):
  """Fills label_folder files with empty label .txt files for frustum-net name consistency with image_folder .png files"""
  existing_label_names = set(map(lambda fn: os.path.splitext(fn)[0], 
                             os.listdir(os.path.join(waymo_path, label_folder))))
  
  required_names = set(map(lambda fn: os.path.splitext(fn)[0], 
                           os.listdir(os.path.join(waymo_path, image_folder))))
  
  for empty_labels in (required_names - existing_label_names):
    open(os.path.join(waymo_path, label_folder, empty_labels + ".txt"), "a").close()
    
fill_with_empty_lables(WAYMO_PATH, "label_2", "image_2")

In [0]:
def test_pngs(path_to_img="/content/drive/My Drive/Waymo_to_Kitti_adapter_train_1/image_2"):
    return all([imghdr.what(os.path.join(path_to_img, img)) == "png" for img in os.listdir(path_to_img)])

In [0]:
FP_sampler.run_detector(1000, plot_det=False, fp_min_pts_num_threshold=30, dest_path="/content/drive/My Drive/PointNet/Data_folder_train_0001/TYPE_FP_PEDESTRIAN")

In [0]:
# Once crashed with: TypeError: long() argument must be a string or a number, not 'PngImageFile'